# Chapter 18: Managing Resources – Garbage Collection & IDisposable

One of the key benefits of .NET is automatic memory management. The **garbage collector (GC)** frees you from manually allocating and deallocating memory, eliminating entire classes of bugs like dangling pointers and memory leaks. However, the GC only manages **managed memory** – memory allocated on the managed heap. Your applications often use **unmanaged resources** such as file handles, network connections, database connections, and GDI+ objects. These resources are not automatically released by the GC and must be handled explicitly.

In this chapter, you'll learn:

- How the **.NET garbage collector** works: generations, mark‑and‑sweep, and compaction.
- The difference between **managed** and **unmanaged resources**.
- The **`IDisposable`** interface and the **`using` statement** for deterministic resource cleanup.
- **Finalizers** (destructors) – what they are, why they exist, and their limitations.
- The **Dispose pattern** – implementing `IDisposable` correctly for classes that own unmanaged resources.
- **`IAsyncDisposable`** and **`await using`** for asynchronous cleanup.
- Best practices and common pitfalls.

By the end, you'll be able to write classes that manage resources cleanly and efficiently, avoiding resource leaks and ensuring your applications are robust.

---

## 18.1 The .NET Garbage Collector (GC)

The GC is a component of the Common Language Runtime (CLR) that automatically reclaims memory occupied by objects that are no longer in use. It operates on the **managed heap**, where all reference‑type objects are allocated.

### How Allocation Works

When you create an object with `new`, it is allocated on the managed heap. Allocation is fast because the heap is just a contiguous block of memory, and the next object is placed right after the previous one (like a stack). The GC keeps a pointer to the next free memory location.

### Generations

To optimize performance, the GC divides objects into **generations** based on their age:

- **Generation 0 (gen0)** – Newly allocated objects. Most objects are short‑lived (e.g., local variables) and are collected here.
- **Generation 1 (gen1)** – Objects that survived a gen0 collection. A buffer between young and old objects.
- **Generation 2 (gen2)** – Objects that survived a gen1 collection. These are long‑lived (e.g., static data, application caches).

The GC collects different generations at different frequencies:

- **Gen0 collections** happen frequently and are fast.
- **Gen1 collections** are less frequent.
- **Gen2 collections** are rare and can be expensive.

This generational approach makes GC efficient: most collections only touch young objects, which are likely garbage.

### When Does GC Happen?

A collection occurs when:

- Gen0 fills up (allocation budget exceeded).
- The system is low on memory.
- You call `GC.Collect()` explicitly (almost never recommended).

### Mark and Sweep

The GC works in phases:

1. **Mark** – Starting from application roots (static fields, local variables on stack, CPU registers, etc.), the GC traverses object references and marks all reachable objects as alive.
2. **Sweep** – The GC reclaims memory from objects that were not marked (garbage). This may involve **compaction** – moving live objects together to make the heap contiguous again, which improves allocation performance and reduces fragmentation.

For gen0 and gen1, compaction is common. Gen2 compaction is more expensive and may be avoided unless necessary.

### Impact on Performance

GC pauses application threads. For most applications, gen0 collections are so fast that the pause is negligible. However, large object heap (LOH) allocations (objects ≥ 85,000 bytes) are handled separately and are not compacted by default, which can lead to fragmentation.

Understanding GC helps you write code that minimizes unnecessary allocations and reduces GC pressure (e.g., reusing objects, using structs appropriately, avoiding large temporary objects in loops).

---

## 18.2 Managed vs. Unmanaged Resources

- **Managed resources** – Objects that are entirely managed by the CLR, such as instances of classes you create. The GC automatically frees their memory when they are no longer referenced.
- **Unmanaged resources** – Operating system resources like file handles, database connections, network sockets, bitmaps, etc. These are not managed by the GC. If you don't release them explicitly, they can be leaked, leading to resource exhaustion.

The GC does **not** manage unmanaged resources. Even if a managed object wrapping an unmanaged resource becomes unreachable, the GC will reclaim the object's memory, but the underlying unmanaged resource may remain open unless the object provides a way to release it. That's where `IDisposable` comes in.

---

## 18.3 The `IDisposable` Interface

`IDisposable` is a simple interface defined in the `System` namespace:

```csharp
public interface IDisposable
{
    void Dispose();
}
```

When a class implements `IDisposable`, it promises to release its unmanaged resources when `Dispose` is called. The consumer of the class should call `Dispose` when they are done with the object.

### Example: Using a `StreamReader`

```csharp
StreamReader reader = new StreamReader("file.txt");
string content = reader.ReadToEnd();
reader.Dispose(); // releases the file handle
```

Forgetting to call `Dispose` can keep the file locked until the GC finalizes the object (which may take a long time). To make this automatic, C# provides the `using` statement.

---

## 18.4 The `using` Statement

The `using` statement ensures that `Dispose` is called even if an exception occurs.

```csharp
using (StreamReader reader = new StreamReader("file.txt"))
{
    string content = reader.ReadToEnd();
    // reader is automatically disposed when exiting the block
}
```

The compiler translates this into a `try`/`finally` block:

```csharp
StreamReader reader = new StreamReader("file.txt");
try
{
    string content = reader.ReadToEnd();
}
finally
{
    if (reader != null)
        reader.Dispose();
}
```

You can declare multiple variables in one `using` statement by nesting:

```csharp
using (StreamReader reader = new StreamReader("file.txt"))
using (StreamWriter writer = new StreamWriter("output.txt"))
{
    // work with both
}
```

### `using` Declaration (C# 8+)

C# 8 introduced **using declarations**, which are simpler: you declare the variable with `using` before its type, and it is disposed at the end of the current scope.

```csharp
using var reader = new StreamReader("file.txt");
string content = reader.ReadToEnd();
// reader disposed here (at the end of the method block)
```

This is especially convenient for reducing nesting.

---

## 18.5 Finalizers (Destructors)

A **finalizer** (also called a destructor in C# syntax) is a special method that the GC calls before reclaiming an object's memory. It is defined like this:

```csharp
class MyClass
{
    ~MyClass()
    {
        // Cleanup code
    }
}
```

Finalizers are intended to release unmanaged resources if the developer forgot to call `Dispose`. However, they have significant drawbacks:

- Finalizers add overhead to the GC (objects with finalizers are placed on a finalization queue and require at least two collections to be reclaimed).
- The exact timing of finalizer execution is non‑deterministic – you cannot predict when it will run.
- Finalizers run on a dedicated thread, and if they block or throw exceptions, they can cause problems.

For these reasons, you should **not** rely on finalizers for normal resource cleanup. Instead, implement the Dispose pattern.

---

## 18.6 The Dispose Pattern

The Dispose pattern is a robust way to implement `IDisposable` for classes that own unmanaged resources. It ensures that:

- `Dispose` releases resources deterministically.
- A finalizer also releases resources if `Dispose` was not called (a safety net).
- Finalization can be suppressed after `Dispose` to improve performance.

Here's the standard pattern:

```csharp
public class ResourceHolder : IDisposable
{
    private bool _disposed = false;
    private IntPtr _unmanagedResource; // Example unmanaged handle
    private Stream _managedResource;   // Example managed resource (also IDisposable)

    public ResourceHolder()
    {
        _unmanagedResource = AllocateUnmanagedResource();
        _managedResource = File.OpenRead("somefile.txt");
    }

    // Public Dispose method
    public void Dispose()
    {
        Dispose(true);
        GC.SuppressFinalize(this); // Tell GC not to call finalizer
    }

    // Protected virtual Dispose method – can be overridden by derived classes
    protected virtual void Dispose(bool disposing)
    {
        if (_disposed)
            return;

        if (disposing)
        {
            // Release managed resources that implement IDisposable
            _managedResource?.Dispose();
        }

        // Release unmanaged resources
        if (_unmanagedResource != IntPtr.Zero)
        {
            FreeUnmanagedResource(_unmanagedResource);
            _unmanagedResource = IntPtr.Zero;
        }

        _disposed = true;
    }

    // Finalizer
    ~ResourceHolder()
    {
        Dispose(false);
    }

    // Optional: throw if disposed
    protected void ThrowIfDisposed()
    {
        if (_disposed)
            throw new ObjectDisposedException(GetType().Name);
    }
}
```

**Key points:**

- The `Dispose(bool disposing)` method centralizes cleanup logic. The `disposing` flag indicates whether we are called from `Dispose` (true) or from the finalizer (false).
- When called from `Dispose` (disposing = true), we release both managed and unmanaged resources.
- When called from the finalizer (disposing = false), we only release unmanaged resources – because managed ones may already be finalized or be referencing objects that are already gone.
- `GC.SuppressFinalize(this)` tells the GC that the finalizer doesn't need to run, improving efficiency.
- The finalizer calls `Dispose(false)` as a safety net.
- The class can include a `ThrowIfDisposed` helper to guard methods after disposal.

Derived classes can override `Dispose(bool)` to release their own resources, remembering to call `base.Dispose(bool)`.

---

## 18.7 `IAsyncDisposable` and `await using`

With asynchronous programming, some cleanup operations (like flushing a stream or closing a database connection) benefit from being asynchronous. .NET Core 3.0 introduced `IAsyncDisposable`:

```csharp
public interface IAsyncDisposable
{
    ValueTask DisposeAsync();
}
```

Classes like `Stream` and `HttpClient` implement this interface. To use it, you can use `await using`:

```csharp
await using (var stream = new FileStream("file.txt", FileMode.Open))
{
    // work with stream
} // DisposeAsync called automatically
```

Or with the declaration form:

```csharp
await using var stream = new FileStream("file.txt", FileMode.Open);
```

The pattern for implementing `IAsyncDisposable` is similar to the synchronous one, but with async disposal:

```csharp
public class AsyncResourceHolder : IAsyncDisposable
{
    private IAsyncDisposable _managedAsyncResource;
    private bool _disposed;

    public async ValueTask DisposeAsync()
    {
        await DisposeAsyncCore();
        GC.SuppressFinalize(this);
        _disposed = true;
    }

    protected virtual async ValueTask DisposeAsyncCore()
    {
        if (_disposed)
            return;

        if (_managedAsyncResource != null)
        {
            await _managedAsyncResource.DisposeAsync();
        }

        // Synchronous cleanup for unmanaged resources
    }

    ~AsyncResourceHolder() => Dispose(false); // fallback
}
```

Often, you'll implement both `IDisposable` and `IAsyncDisposable` if you have both synchronous and asynchronous consumers.

---

## 18.8 Putting It All Together: A Custom FileWriter Class

Let's build a simple class that manages a file handle and implements the Dispose pattern correctly.

```csharp
using System;
using System.IO;
using System.Text;
using System.Threading.Tasks;

namespace ResourceManagementDemo
{
    public class FileWriter : IDisposable, IAsyncDisposable
    {
        private FileStream _fileStream;
        private StreamWriter _streamWriter;
        private bool _disposed;

        public FileWriter(string filePath)
        {
            _fileStream = new FileStream(filePath, FileMode.Create, FileAccess.Write);
            _streamWriter = new StreamWriter(_fileStream, Encoding.UTF8);
        }

        public void Write(string text)
        {
            ThrowIfDisposed();
            _streamWriter.Write(text);
        }

        public async Task WriteAsync(string text)
        {
            ThrowIfDisposed();
            await _streamWriter.WriteAsync(text);
        }

        public void Flush()
        {
            ThrowIfDisposed();
            _streamWriter.Flush();
        }

        // Synchronous Dispose
        public void Dispose()
        {
            Dispose(true);
            GC.SuppressFinalize(this);
        }

        // Asynchronous Dispose
        public async ValueTask DisposeAsync()
        {
            await DisposeAsyncCore();
            Dispose(false); // clean up managed resources synchronously if needed
            GC.SuppressFinalize(this);
        }

        protected virtual void Dispose(bool disposing)
        {
            if (_disposed)
                return;

            if (disposing)
            {
                // Dispose managed resources
                _streamWriter?.Dispose();
                _fileStream?.Dispose();
            }

            // No unmanaged resources to release here
            _disposed = true;
        }

        protected virtual async ValueTask DisposeAsyncCore()
        {
            if (_disposed)
                return;

            if (_streamWriter != null)
            {
                await _streamWriter.DisposeAsync();
            }
            if (_fileStream != null)
            {
                await _fileStream.DisposeAsync();
            }

            _disposed = true;
        }

        ~FileWriter()
        {
            Dispose(false);
        }

        private void ThrowIfDisposed()
        {
            if (_disposed)
                throw new ObjectDisposedException(nameof(FileWriter));
        }
    }

    class Program
    {
        static async Task Main()
        {
            // Using synchronous disposal
            using (var writer = new FileWriter("test.txt"))
            {
                writer.Write("Hello, world!");
                writer.Flush();
            }

            // Using asynchronous disposal
            await using (var writer = new FileWriter("test2.txt"))
            {
                await writer.WriteAsync("Hello, async world!");
            }

            Console.WriteLine("Done.");
        }
    }
}
```

**Explanation:**

- `FileWriter` encapsulates a `FileStream` and `StreamWriter`. Both are `IDisposable` and `IAsyncDisposable`.
- It implements both interfaces, providing both synchronous and asynchronous disposal.
- The `Dispose(bool)` method handles managed resource cleanup when `disposing` is true.
- `DisposeAsyncCore` handles asynchronous cleanup.
- A finalizer calls `Dispose(false)` as a safety net.
- The class guards all public methods with `ThrowIfDisposed`.

This pattern ensures that resources are released deterministically when the consumer uses `using` or `await using`, and even if they forget, the finalizer will eventually release them (though with a delay).

---

## 18.9 Best Practices and Common Pitfalls

### 1. Always Call Dispose (or Use `using`)

If a class implements `IDisposable`, you should call `Dispose` when you're done. The easiest way is with a `using` statement or declaration.

### 2. Implement the Dispose Pattern for Classes Owning Unmanaged Resources

If your class directly owns an unmanaged resource (e.g., a handle from `Marshal.AllocHGlobal`), implement the full pattern. If it only owns other managed `IDisposable` objects, you may not need a finalizer (because those objects have their own finalizers), but you should still implement `IDisposable` to allow deterministic cleanup.

### 3. Seal the `Dispose(bool)` Method if Your Class Is Sealed

If your class is `sealed`, you can make `Dispose(bool)` private and non‑virtual.

### 4. Avoid Finalizers Unless Necessary

Finalizers add overhead and delay garbage collection. Only implement one if you directly hold an unmanaged resource that you must guarantee release. For classes that only compose other `IDisposable` objects, rely on their finalizers.

### 5. Suppress Finalization After Dispose

Always call `GC.SuppressFinalize(this)` in `Dispose()` to prevent the finalizer from running.

### 6. Make `Dispose` Idempotent

Calling `Dispose` multiple times should not throw an exception. Use a flag to track disposal.

### 7. Throw `ObjectDisposedException` When Accessing a Disposed Object

After disposal, public members should throw to indicate the object is no longer usable.

### 8. Consider Providing Both Synchronous and Asynchronous Disposal

If your class is likely to be used in async scenarios, implement `IAsyncDisposable` as well.

### 9. Be Careful with Static References

Static fields can keep objects alive indefinitely, preventing garbage collection and causing memory leaks.

### 10. Use `GC.Collect` Only in Rare Circumstances

For almost all production code, you should not call `GC.Collect()`. The GC is self‑tuning. Manual collection often hurts performance.

---

## 18.10 Chapter Summary

In this chapter, you've learned about resource management in .NET:

- The **garbage collector** automatically reclaims managed memory using generational mark‑and‑sweep.
- **Unmanaged resources** (file handles, network connections, etc.) must be released explicitly.
- **`IDisposable`** and the **`using` statement** provide deterministic cleanup.
- **Finalizers** act as a safety net for unmanaged resources but have performance costs.
- The **Dispose pattern** ensures proper cleanup in inheritance hierarchies and protects against resource leaks.
- **`IAsyncDisposable`** and **`await using`** extend the pattern to asynchronous scenarios.
- A practical example demonstrated how to implement both synchronous and asynchronous disposal correctly.

Proper resource management is crucial for building reliable, scalable applications. With these tools, you can ensure that your code never leaks resources and performs well under load.

In the next chapter, **File I/O and Serialization**, we'll explore how to work with files and directories, read and write data, and serialize objects to formats like JSON and XML.

**Exercises:**

1. Write a class `TempFile` that creates a temporary file on disk and deletes it when disposed. Implement `IDisposable`.
2. Create a class `DatabaseConnection` that wraps a `SqlConnection` (which implements `IDisposable`). Implement `IDisposable` correctly (without a finalizer).
3. Modify the `FileWriter` example to also include an unmanaged resource (e.g., a Windows file handle obtained via `CreateFile` from kernel32). Implement the full Dispose pattern with a finalizer.
4. Write a program that uses `Stopwatch` to measure the performance difference between calling `Dispose` and relying on finalization. (Hint: create many objects that allocate unmanaged resources and see how long it takes for memory to be reclaimed.)
5. Experiment with `await using`: create a class that implements `IAsyncDisposable` and simulate asynchronous cleanup with `Task.Delay`.

Now, get ready to dive into file I/O and serialization in Chapter 19!

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../3. advanced_language_features/17. pattern_matching_in_depth.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='19. file_io_and_serialization.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
